In [1]:
import numpy as np

In [12]:
def g(p, beta):
    """
    Implements the Potts model tree recursion.
    
    Inputs: 
    p: a (d x q) matrix, giving the marginal probabilities for each of the children. 
    d here is the number of children and q the number of colors.
    
    beta: the temperature.
    
    
    Output: a row vector in R^q, giving the marginal probabilities of the parent.
    """
    numerator = np.prod(1 - (1-beta) * p, axis=0)
    denominator = np.sum(numerator)
    return numerator / denominator

In [73]:
def J(p, beta, Psi):
    """
    Calculates the Jacobian of the tree recursion.
    
    Inputs:
    p: a (d x q) matrix, giving the marginal probabilities for each of the children.
    d here is the number of children and q the number of colors.

    beta: the temperature.
    
    Psi: (a python function) the derivative of the potential function. It's important 
    that this function is implemented in such a way that in can be applied elementwise
    to numpy arrays.
    
    Output: a (d x q x q) 3-tensor J, that represents the Jacobian of the tree recursion. 
    For each i \in [d], J_i is the Jacobian with respect to just the i'th child's marginals.
    """
    term1 = (1-beta) * ( np.outer(g(p, beta), g(p, beta)) - np.diag(g(p, beta)) )
    
    def to_apply(p_i):
        return np.diag(Psi(g(p,beta))) @ term1 @ np.diag(1/(1-(1-beta)*p_i)) @ np.diag(1/Psi(p_i))
    
    return np.apply_along_axis(to_apply, 1, p)

In [74]:
p = np.array([[0.1,0.9], [0.3,0.7], [0.1,0.9]])

In [76]:
def Psi(x):
    return 1/(np.sqrt(x) * (1-x))

J(p, 0, Psi)

array([[[-0.31539449,  0.94618347],
        [ 0.02294157, -0.06882472]],

       [[-0.54627928,  0.83445538],
        [ 0.03973597, -0.0606977 ]],

       [[-0.31539449,  0.94618347],
        [ 0.02294157, -0.06882472]]])

array([[9.75758477e-01, 1.20464010e-02],
       [1.20464010e-02, 1.48720999e-04]])